# SQL Server Integration Server 

### Import CSV Files into SQL Server Tables Using SSIS
- create a flat file connection in the connection manger
- create source flat file and map it to the flat file connection
- create a connection to the database server
    - in the connection manger > data links select the database or write the database name in the initial catalogue field
- create destination database and map it to the database connection
- in the control flow add an execute sql task on the database 
    - using delete from to truncate all row before the initial population (truncate can work if there is no foreign key constraint)
    - when dealing with identity columns it is not allowed to add data into them so we ignore id columns
    - testing the integration would increment the identity id column, we use DBCC CHECKIDENT on the table to reset the count 
    ```sql
    DELETE FROM institutions.Institutions;
    DBCC CHECKIDENT ('Institutions.Institutions', RESEED, 0);
    ``` 

> When creating the connection use __Microsoft OLE DB Driver for SQL Server__ `provider` in the `OLE DB` `connection manger` on __vs studio 22__ with __SSMS 19__. tutorials use older versions of vs studio and ssms that support the Native provider `SQLNCLI11`. `The specified provider is not supported. Please choose different provider in connection manager.` error occurs when using the default provider. [Microsoft announcement link](https://learn.microsoft.com/en-us/sql/relational-databases/native-client/sql-server-native-client?view=sql-server-ver15)

### Insert Order 

#### dataflow task 1
01. Institutions_Institutions
02. Institutions_Semesters
03. Institutions_TimeSlots
04. Institution_Programs
05. Institutions_Departments
06. Institutions_Instructors (leave instructor_institution_department_id null this step)
#### dataflow task 2
07. Institutions_ClassRooms
08. Institutions_InstitutionPrograms
09. Institutions_InstitutionDepartments
10. Institutions.Instructors (import institution_department_id column)
#### dataflow task 3
11. Programs.Courses  
12. Programs.Assessments
#### dataflow task 4
13. Programs.CoursePrerequisites
14. Programs.Topic
15. Programs.Exams
#### dataflow task 5
16. Programs.Questions
#### dataflow task 6
17. Programs.Answers 
18. Programs.ProgramCourses
19. Programs.CourseSemester
20. Institution.ClassSchedule
21. Students.Students
#### dataflow task 7
22. Students.StudentEnrollments
23. Students.AssessmentScores
24. Students.CourseResults
25. Students.ExamSubmission


### Remove All Rows From All Tables
```sql 
USE LMS;
GO

-- 25-ExamSubmissions
DELETE FROM Students.ExamSubmissions;

-- 24-CourseResults
DELETE FROM Students.CourseResults;

-- 23-AssessmentScores
DELETE FROM Students.AssessmentScores;

-- 22-StudentEnrollments
DELETE FROM Students.StudentEnrollments;

-- 21-Students
DELETE FROM Students.Students;

-- 20-ClassSchedule 
DELETE FROM Institutions.ClassSchedule;

-- 19-CourseSemester
DELETE FROM Programs.CourseSemester;

-- 18-ProgramsCourses
DELETE FROM Programs.ProgramCourses;

-- 17-Answers
DELETE FROM Programs.Answers;

-- 16-Questions
DELETE FROM Programs.Questions;

-- 15-Exams
DELETE FROM Programs.Exams;

-- 14-Topics
DELETE FROM Programs.Topics;

-- 13-CoursePrerequisites
DELETE FROM Programs.CoursePrerequisites;

-- 12-Assessments
DELETE FROM Programs.Assessments;

-- 11-Courses
DELETE FROM Programs.Courses;

-- 10-Instructors (institutions_institution_department_id)
UPDATE Institutions.Instructors
SET instructor_institution_department_id = NULL;

-- 9- InstitutionsDepartments
DELETE FROM Institutions.InstitutionDepartments;

-- 8- InstitutionsPrograms
DELETE FROM Institutions.InstitutionPrograms;

-- 7-Classrooms
DELETE FROM Institutions.ClassRooms;

-- 6-Instructors
DELETE FROM Institutions.Instructors;

-- 5-Departments
DELETE FROM Institutions.Departments;

-- 4-Programs 
DELETE FROM Institutions.Programs;

-- 3-TimeSlots
DELETE FROM Institutions.TimeSlots;

-- 2-Semesters
DELETE FROM Institutions.Semesters;

-- 1-Institutions
DELETE FROM Institutions.Institutions;
```

### Reset Identity Columns
```sql
USE LMS;
GO

-- A. Institutions --
DBCC CHECKIDENT('Institutions.Institutions', RESEED, 0);
DBCC CHECKIDENT('Institutions.Semesters', RESEED, 0);
DBCC CHECKIDENT('Institutions.TimeSlots', RESEED, 0);
DBCC CHECKIDENT('Institutions.Programs', RESEED, 0);
DBCC CHECKIDENT('Institutions.Departments', RESEED, 0);
DBCC CHECKIDENT('Institutions.Instructors', RESEED, 0);
DBCC CHECKIDENT('Institutions.ClassRooms', RESEED, 0);
DBCC CHECKIDENT('Institutions.InstitutionPrograms', RESEED, 0);
DBCC CHECKIDENT('Institutions.InstitutionDepartments', RESEED,0);

-- B. Programs --
DBCC CHECKIDENT('Programs.ProgramCourses', RESEED,0);
DBCC CHECKIDENT('Programs.Courses', RESEED,0);
DBCC CHECKIDENT('Programs.Assessments', RESEED,0);
DBCC CHECKIDENT('Programs.Topics', RESEED,0);
DBCC CHECKIDENT('Programs.Exams', RESEED,0);
DBCC CHECKIDENT('Programs.Questions', RESEED,0);
DBCC CHECKIDENT('Programs.Answers', RESEED,0);

-- C. INSTITUTIONS --
DBCC CHECKIDENT('Institutions.ClassSchedule', RESEED,0);

-- D. STUDENTS --
DBCC CHECKIDENT('Students.Students', RESEED,0);
```

### Disable Foreign key Constraint Temporally
```sql
USE LMS;
GO
ALTER TABLE Institutions.Instructors NOCHECK CONSTRAINT fk_instructor_supervisor

ALTER TABLE Institutions.Instructors NOCHECK CONSTRAINT fk_instructor_institution_department

USE LMS;
GO
ALTER TABLE Institutions.Instructors WITH CHECK CHECK CONSTRAINT fk_instructor_supervisor


USE LMS;
GO
ALTER TABLE Institutions.Instructors WITH CHECK CHECK CONSTRAINT fk_instructor_institution_department
```